In [1]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")

# Initialize video capture (replace 'input_video.mp4' with your video file or 0 for webcam)
cap = cv2.VideoCapture('new.mp4')

# Create VideoWriter to save output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for the output video (XVID is a commonly used codec)
output_video = cv2.VideoWriter('output_video_with_boxes.avi', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Use YOLOv8 to detect objects in the current frame
    results = model(frame, verbose=False)
    detections = results[0].boxes.data.cpu().numpy()  # Extract detections

    for detection in detections:
        # Extract bounding box coordinates and class information
        x1, y1, x2, y2, confidence, class_id = detection[:6]

        # Draw bounding box on the frame
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Write the processed frame to the output video
    output_video.write(frame)

    # Display the video feed with bounding boxes
    cv2.imshow("YOLOv8 Object Detection", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
output_video.release()
cv2.destroyAllWindows()


object with time stamp

In [1]:
import cv2
from ultralytics import YOLO
import time

# Load the YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")  # Use 'yolov8n.pt' for a lightweight model, or 'yolov8m.pt'/'yolov8l.pt' for larger models.

# Initialize video capture (replace 0 with a video file path for offline analysis)
cap = cv2.VideoCapture('traffic-mini.mp4')  # Use 0 for webcam, or provide a video file path like "video.mp4"

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = 0

# List to store timestamps for detected objects
detected_timestamps = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Use YOLOv8 to detect objects in the current frame
    results = model(frame, verbose=False)
    detections = results[0].boxes.data.cpu().numpy()  # Extract detections

    for detection in detections:
        # Extract bounding box, confidence, and class ID
        x1, y1, x2, y2, confidence, class_id = detection[:6]
        label = results[0].names[int(class_id)]  # Get class name

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Record timestamp of detection
        timestamp = frame_count / fps
        detected_timestamps.append((label, timestamp))

    # Display the video feed with detections
    cv2.imshow("YOLOv8 Object Detection", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print detected objects with timestamps
for obj, ts in detected_timestamps:
    print(f"Object: {obj}, Timestamp: {ts:.2f} seconds")

cap.release()
cv2.destroyAllWindows()


Object: bus, Timestamp: 0.03 seconds
Object: bus, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: person, Timestamp: 0.03 seconds
Object: person, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: motorcycle, Timestamp: 0.03 seconds
Object: person, Timestamp: 0.03 seconds
Object: bus, Timestamp: 0.03 seconds
Object: person, Timestamp: 0.03 seconds
Object: person, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: bicycle, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: person, Timestamp: 0.03 seconds
Object: car, Timestamp: 0.03 seconds
Object: bus, Timestamp: 0.07 seconds
Object: bus, Timestamp: 0.07 seconds
Object: car, Timestamp: 0.07 seconds
Object: person, Timestamp: 0.07 seconds
Object: car, Timestamp: 0.07 seconds
Object: person, Timestamp: 0.07 seconds
Obj

object with unique count


In [5]:
import cv2
from ultralytics import YOLO
import numpy as np
from collections import defaultdict

# Load the YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")

# Initialize video capture (replace with 0 for webcam or a file path for a video)
cap = cv2.VideoCapture('traffic-mini.mp4')

# Function to check if two bounding boxes overlap significantly
def is_same_object(box1, box2, threshold=0.5):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Calculate intersection area
    xi_min = max(x1_min, x2_min)
    yi_min = max(y1_min, y2_min)
    xi_max = min(x1_max, x2_max)
    yi_max = min(y1_max, y2_max)

    if xi_max < xi_min or yi_max < yi_min:
        return False

    intersection = (xi_max - xi_min) * (yi_max - yi_min)
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)

    # Calculate IoU (Intersection over Union)
    iou = intersection / float(area1 + area2 - intersection)
    return iou > threshold

# Dictionary to store unique objects by class
object_counts = defaultdict(int)
tracked_objects = defaultdict(list)  # Tracks bounding boxes for each class

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Use YOLOv8 to detect objects in the current frame
    results = model(frame, verbose=False)
    detections = results[0].boxes.data.cpu().numpy()  # Extract detections

    for detection in detections:
        # Extract bounding box, confidence, and class ID
        x1, y1, x2, y2, confidence, class_id = detection[:6]
        label = results[0].names[int(class_id)]  # Get class name

        # Check if this object is already tracked
        current_box = (x1, y1, x2, y2)
        already_tracked = any(is_same_object(current_box, box) for box in tracked_objects[label])

        if not already_tracked:
            # Add to tracked objects and increment the count
            tracked_objects[label].append(current_box)
            object_counts[label] += 1

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the video feed with detections
    cv2.imshow("YOLOv8 Object Detection", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print unique object counts by class
print("\n=== Unique Object Counts ===")
for obj_class, count in object_counts.items():
    print(f"{obj_class}: {count}")

cap.release()
cv2.destroyAllWindows()



=== Unique Object Counts ===
bus: 27
car: 83
person: 80
motorcycle: 8
bicycle: 14
truck: 2


Webcam


In [2]:
from ultralytics import YOLO
import cv2

# Load YOLOv8 model (use 'yolov8n.pt' for faster performance)
model = YOLO('yolov8n.pt')

# Open webcam (or replace 0 with the video file path)
video = cv2.VideoCapture(0)  # 0 = Default webcam; change to 1 if you have multiple webcams

# Check if the webcam is opened correctly
if not video.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Read a frame from the webcam
    ret, frame = video.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Perform object detection
    results = model(frame)  # Pass the frame through the model

    # Annotate the frame with detected objects
    annotated_frame = results[0].plot()

    # Display the annotated frame
    cv2.imshow("Live Object Detection", annotated_frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 1 tv, 395.5ms
Speed: 92.6ms preprocess, 395.5ms inference, 51.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 213.0ms
Speed: 6.0ms preprocess, 213.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 209.7ms
Speed: 4.1ms preprocess, 209.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 208.5ms
Speed: 4.0ms preprocess, 208.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 198.0ms
Speed: 5.0ms preprocess, 198.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 203.6ms
Speed: 5.0ms preprocess, 203.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 225.4ms
Speed: 4.0ms preprocess, 225.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 193.9ms
Speed: 4.3ms preprocess, 193.9ms inf

mobile feed


In [2]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")  # Use 'yolov8m.pt' or 'yolov8l.pt' for larger models if needed.

# Replace with your IP Webcam stream URL
ip_webcam_url = "http://192.168.29.204:8080/video"

# Initialize the video capture from IP Webcam
cap = cv2.VideoCapture(ip_webcam_url)

if not cap.isOpened():
    print("Error: Unable to access the IP Webcam feed!")
    exit()

# Dictionary to count unique objects detected
object_count = {}

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame from IP Webcam feed!")
        break

    # Use YOLOv8 to detect objects in the current frame
    results = model(frame, verbose=False)
    detections = results[0].boxes.data.cpu().numpy()  # Extract detections

    for detection in detections:
        # Extract bounding box, confidence, and class ID
        x1, y1, x2, y2, confidence, class_id = detection[:6]
        label = results[0].names[int(class_id)]  # Get class name

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Count detected objects (unique labels)
        if label in object_count:
            object_count[label] += 1
        else:
            object_count[label] = 1

    # Display the video feed with detections
    cv2.imshow("YOLOv8 Object Detection - IP Webcam", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print total counts of each object detected
print("Total object counts:", object_count)

cap.release()
cv2.destroyAllWindows()


Total object counts: {'person': 1323, 'scissors': 65, 'chair': 7, 'toothbrush': 112, 'skateboard': 2, 'backpack': 14, 'handbag': 28, 'remote': 21, 'cell phone': 143, 'hot dog': 4, 'cat': 2, 'laptop': 23, 'book': 105, 'dog': 5, 'clock': 1, 'bottle': 56, 'tie': 18, 'banana': 2, 'bird': 2}


In [3]:
from ultralytics import YOLO
import cv2

# Load YOLOv8 model (use 'yolov8n.pt' for faster performance)
model = YOLO('yolov8n.pt')

# Open webcam (or replace 0 with the video file path)
video = cv2.VideoCapture(0)  # 0 = Default webcam; change to 1 if you have multiple webcams

# Check if the webcam is opened correctly
if not video.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Set to store unique object classes
unique_classes_detected = set()

while True:
    # Read a frame from the webcam
    ret, frame = video.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Perform object detection
    results = model(frame)  # Pass the frame through the model

    # Annotate the frame with detected objects
    annotated_frame = results[0].plot()

    # Extract object information
    for obj in results[0].boxes.data:  # Get bounding box data
        x1, y1, x2, y2, confidence, class_id = obj[:6]
        class_name = model.names[int(class_id)]  # Convert class_id to class name

        # Add the class to the set of unique objects
        if class_name not in unique_classes_detected:
            unique_classes_detected.add(class_name)
            print(f"Detected new unique object: {class_name}")
    
    # Annotate frame with the list of unique object classes
    unique_classes_text = ", ".join(unique_classes_detected)
    cv2.putText(
        annotated_frame,
        f"Unique Objects: {unique_classes_text}",
        (10, 30),  # Position of the text
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 255, 0),
        2,
    )

    # Display the annotated frame
    cv2.imshow("Live Object Detection", annotated_frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 262.2ms
Speed: 7.0ms preprocess, 262.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Detected new unique object: person

0: 480x640 1 person, 282.6ms
Speed: 6.0ms preprocess, 282.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 250.7ms
Speed: 4.1ms preprocess, 250.7ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 229.6ms
Speed: 3.0ms preprocess, 229.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 213.4ms
Speed: 5.0ms preprocess, 213.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 194.6ms
Speed: 3.0ms preprocess, 194.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 189.3ms
Speed: 4.0ms preprocess, 189.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 179.2ms
Speed: 4.0ms preprocess, 179.2ms infere

In [4]:
from ultralytics import YOLO
import cv2
import winsound  # For sound alert (Windows only)

# Load YOLOv8 model (use 'yolov8n.pt' for faster performance)
model = YOLO('yolov8n.pt')

# Open webcam (or replace 0 with the video file path)
video = cv2.VideoCapture(0)  # 0 = Default webcam; change to 1 if you have multiple webcams

# Check if the webcam is opened correctly
if not video.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Read a frame from the webcam
    ret, frame = video.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Perform object detection
    results = model(frame)  # Pass the frame through the model

    # Annotate the frame with detected objects
    annotated_frame = results[0].plot()

    # Check if a person is detected
    person_detected = False
    for obj in results[0].boxes.data:  # Get bounding box data
        x1, y1, x2, y2, confidence, class_id = obj[:6]
        class_name = model.names[int(class_id)]  # Convert class_id to class name

        if class_name == "person":
            person_detected = True
            break

    # If a person is detected, trigger an alert
    if person_detected:
        cv2.putText(
            annotated_frame,
            "ALERT: Person Detected!",
            (10, 50),  # Position of the text
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 0, 255),  # Red color for alert
            2,
        )
        # Play a sound alert
        winsound.Beep(1000, 500)  # Beep at 1000 Hz for 500 ms

    # Display the annotated frame
    cv2.imshow("Area Monitoring", annotated_frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 287.7ms
Speed: 20.4ms preprocess, 287.7ms inference, 4.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 248.5ms
Speed: 5.0ms preprocess, 248.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.8ms
Speed: 5.2ms preprocess, 201.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 190.2ms
Speed: 5.0ms preprocess, 190.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 202.7ms
Speed: 4.0ms preprocess, 202.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.0ms
Speed: 4.0ms preprocess, 201.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 208.3ms
Speed: 4.5ms preprocess, 208.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 179.2ms
Speed: 4.0ms prep